# 피처엔지니어링 1: 산업코드로 업종중분류, 대분류 만들기

현재 데이터셋에 '업종' feature가 있다. 그러나 이 feature는 산업군 정도만 나태나고 있기 때문에, 기업의 상세 업종을 잘 나타내지 못한다.

기업의 휴폐업에 업종의 상세 내용도 영향을 끼칠 것이라고 생각하기 때문에 산업코드로 업종중분류, 대분류를 만들어보려 한다.

산업코드1, 산업코드2, 산업코드3이 있으나 산업코드2, 3은 추후 추가된 것이기 때문에 가장 기본이 되는 산업코드1을 기준으로 업종중분류, 대분류를 만든다.

산업코드1에 결측치가 있는 경우 나이스 비즈인포에서 크롤링하여 결측치를 보간하였다.

In [2]:
import pandas as pd
import numpy as np

import json
import re

In [3]:
df = pd.read_csv('./custom_data/active_closed_all.csv', encoding='cp949')

In [4]:
# 크롤링한 산업코드1의 결측치를 가지는 기업 정보, 산업코드 json 파일 불러오기
with open ("./custom_data/indurstry_code_missing_value.json", "r") as f:
    crawling_data = json.load(f)

>찾은 결과

[그냥 안 나오는 기업]
- '블루인더스', => 마스크 파는 곳, 정천식으로 대표자가 바뀜, 사업자등록번호가 다르다
- '한서공조', => 냉난방공조 메이커 회사, 사업자등록번호가 다르다
- '코단에코크리트', => 콘크리트블록 파는 곳, 사업자등록번호가 다르다
- '에프티이앤이', 주식회사 라임의 자회사, 마스크 파는 곳, 실제 대표자명도 다르고 사업자등록번호도 다르다
- '비티캠퍼스', => 차량번호 인식하는 제품 파는 곳, 사업자등록번호가 다르다, 이름도 비티캠퍼스가 아니라 비티컴퍼스임
- '에이비온바이오메드', => 19년 3월 15일 에이비온이 바이오메드사업부(시약 및 의약품 수입, 도소매업)의 영업을 중지함

[지점이라서 안 나오는 기업]
- '엘에프네트웍스송림아울렛점',
- '대지이앤씨산업(주)인천동부지점',
- '성원디앤에프부평지점',
- '세종에스엠에스화산지점',
- '한국교육시스템(주)분당지점코레샵',
- '세종에스엠에스한국관광용품센타지점',
- '삼진아트팩지점',
- '대신네트웍스(주)춘천지점',
- '쎄니트해운대지점',
- '블루네트웍스중부지사',
- '범신사세종지사',
- '에코로화성지점',
- '부농 군산지점',
- '회명애쉬랜드(주)군산',
- '신창테크전주지점',
- '예건광주지점',
- '엔도비전대구지점',
- '성우아이디건설동대구지점',
- '더블유아이케이중부',
- '포스토피아대구센터',
- '네비엔영천사업소',
- '시소아이티부산지사',
- '가양김해지점',
- '포스토피아제주센터',
- '네티브(주)부산지사',
- '정선레미콘남서울공장',
- '코레일관광개발(주)강빛마을펜션'
- '신한정밀공업(주)대전공장'

In [5]:
# 크롤링으로 안 나오는 경우에 대해 수작업으로 수집한 데이터
search_list = ['(C13229)기타 직물제품 제조업', '(G46599)그 외 기타 기계 및 장비 도매업', '(C23239)기타 건축용 비내화 요업제품 제조업', '(M72122)환경 관련 엔지니어링 서비스업',
               '(A13213)합성섬유 제조업', '(C26310)컴퓨터 제조업', '(M70113)의학 및 약학 연구개발업', '(L68112)비주거용 건물 임대업', 
               '(F42201)배관 및 냉ㆍ난방 공사업', '(F42492)건물용 금속공작물 설치 공사업', '(I56111)한식 일반 음식점업', '(C33401)인형 및 장난감 제조업', 
               '(I56111)한식 일반 음식점업', '(C17221)종이 포대 및 가방 제조업', '(F42322)내부 통신배선 공사업', '(C24122)냉간 압연 및 압출 제품 제조업', 
               '(S95120)통신장비 수리업', '(C18111)경 인쇄업', '(C32091)금속 가구 제조업', '(C20313)유기질 비료 및 상토 제조업', '(C20495)바이오 연료 및 혼합물 제조업', 
               '(C25111)금속 문, 창, 셔터 및 관련제품 제조업','(F41100)건물 건설업','(C27199)그 외 기타 의료용 기기 제조업','(F41110)주거용 건물 건설업',
               '(E38230)건설 폐기물 처리업','(J63111)자료 처리업','(G45212)자동차용 전용 신품 부품 판매업','(J58221)시스템 소프트웨어 개발 및 공급업','(L68121)주거용 건물 개발 및 공급업',
               '(J63111)자료 처리업','(J58222)응용 소프트웨어 개발 및 공급업','(C23322)레미콘 제조업','(N75210)여행사업', '(C29294)주형 및 금형 제조업',
               '(F41110)주거용 건물 건설업']
               
crawling_data[1]["기업명"] = ['블루인더스', '한서공조', '코단에코크리트', '에프티이앤이', '비티캠퍼스', '에이비온바이오메드', '엘에프네트웍스송림아울렛점', '이앤이솔루션',
                           '대지이앤씨산업(주)인천동부지점', '성원디앤에프부평지점', '세종에스엠에스화산지점', '한국교육시스템(주)분당지점코레샵', '세종에스엠에스한국관광용품센타지점','삼진아트팩지점', '대신네트웍스(주)춘천지점', '쎄니트해운대지점', '블루네트웍스중부지사', '범신사세종지사', '에코로화성지점', '부농 군산지점', 
                           '회명애쉬랜드(주)군산', '신창테크전주지점','예건광주지점','엔도비전대구지점','성우아이디건설동대구지점','더블유아이케이중부','포스토피아대구센터','네비엔영천사업소','시소아이티부산지사','가양김해지점','포스토피아제주센터','네티브(주)부산지사','정선레미콘남서울공장','코레일관광개발(주)강빛마을펜션', '신한정밀공업(주)대전공장']

crawling_data[1]["산업코드"] = []
crawling_data[1]["주요사업내용"] = []

for cor_info in search_list:
    indurstry_code = re.findall('\(([^)]+)', cor_info)[0][1:]   # 산업코드 추출
    business_details = re.sub('\(.*\)|\s-\s.*', '',cor_info) # 주요사업내용 추출
    crawling_data[1]["산업코드"].append(indurstry_code)
    crawling_data[1]["주요사업내용"].append(business_details)

In [6]:
# 크롤링으로 수집한 데이터에 수작업으로 찾은 데이터를 합치기
crawling_data[0]["사업자등록번호"].extend(crawling_data[1]["사업자등록번호"] )
crawling_data[0]["기업명"].extend(crawling_data[1]["기업명"] )
crawling_data[0]["산업코드1"].extend(crawling_data[1]["산업코드"] )
crawling_data[0]["주요사업내용"].extend(crawling_data[1]["주요사업내용"] )

In [20]:
# 산업코드에 결측치가 있는 행, 열을 찾아서 값 보간하기
for cor_num, cor_name, cor_code, cor_indurstry in zip(crawling_data[0]["사업자등록번호"], crawling_data[0]["기업명"], crawling_data[0]["산업코드1"], crawling_data[0]["주요사업내용"]):
    new_cor_name = re.sub('\(.*\)|\s-\s.*', '',cor_name)
    idx = df[df['기업명'] == cor_name].index
    new_idx = df[df['기업명'] == new_cor_name].index
    if len(new_idx) != 0:
        idx = new_idx
    for i in idx:
        df.loc[i, '산업코드1'] = cor_code
        df.loc[i, '주요사업내용'] = cor_indurstry

In [22]:
# 업종중분류, 대분류를 만들기 위해 한국표준산업분류10차 표 데이터 불러오기
indurstry_code_df = pd.read_excel('./custom_data/한국표준산업분류10차_표.xlsx')

In [23]:
# 잘못되어 있는 column명 제거
indurstry_code_middle_name = indurstry_code_df[['Unnamed: 2', 'Unnamed: 3']].dropna().reset_index(drop=True)
indurstry_code_main_name = indurstry_code_df[['Unnamed: 1']].dropna().reset_index(drop=True)

In [24]:
# 잘못되어 있는 column명 변경
indurstry_code_middle_name = indurstry_code_middle_name.rename(columns=indurstry_code_middle_name.iloc[0])
indurstry_code_middle_name_change_columns = indurstry_code_middle_name.drop(indurstry_code_middle_name.index[0])

indurstry_code_main_name = indurstry_code_main_name.rename(columns=indurstry_code_main_name.iloc[0])
indurstry_code_main_name_change_columns = indurstry_code_main_name.drop(indurstry_code_main_name.index[0])

In [25]:
# 80에 대한 산업분류가 나와있지 않아 수작업으로 찾은 결과 추가하기
add_data = pd.DataFrame({'코드':['80'], '항목명':['교육 서비스업']})
indurstry_code_middle_name_final = pd.concat([indurstry_code_middle_name_change_columns, add_data], ignore_index = True)

In [26]:
def transfer_main_class(value):
    """
    항목명에서 산업코드의 범위를 추출하는 함수 입니다.
    ----------------------------------------------
    input = 항목명(문자열)
    return = 산업코드 범위(정수 리스트)
    """

    code_range = re.findall('\(([^)]+)', value)[0]
    item_name = re.sub('\(.*\)|\s-\s.*', '',value)

    code_range_list = code_range.split('~')
    if len(code_range_list) == 2:
        return list(range(int(code_range_list[0]), int(code_range_list[1])+1)), item_name
    else:
        return list(map(int, code_range_list)), item_name

indurstry_code_main_name_add_columns = indurstry_code_main_name_change_columns.copy()
indurstry_code_main_name_add_columns[['코드', '항목명']] = indurstry_code_main_name_change_columns\
                                                        .apply(lambda x:transfer_main_class(x['항목명']), axis=1, result_type='expand')

In [27]:
# 80에 대한 산업분류가 나와있지 않아 수작업으로 찾은 결과 추가하기
add_data = pd.DataFrame({'코드':[[80]], '항목명':['교육 서비스업']})
indurstry_code_main_name_final = pd.concat([indurstry_code_main_name_add_columns, add_data], ignore_index = True)

In [28]:
middle_dict = indurstry_code_middle_name_final.to_dict()
main_dict = indurstry_code_main_name_final.to_dict()

In [29]:
def add_middle_class(value):
    """
    산업코드 앞 2자리를 한국표준산업분류 10차를 기준으로 중분류로 변경하는 함수입니다.
    ---------------------------------------
    input = 산업코드1(실수형)
    retrun  = 업종중분류(문자열)
    ---------------------------------------
    """
    value = str(value)[:2]
    indurstry_code_key = [k for k, v in middle_dict['코드'].items() if v == value][0]
    
    return middle_dict['항목명'][indurstry_code_key]

In [30]:
def add_main_class(value):
    """
    산업코드 앞 2자리를 한국표준산업분류 10차를 기준으로 대분류로 변경하는 함수입니다.
    ---------------------------------------
    input = 산업코드1(실수형)
    retrun  = 업종대분류(문자열)
    ---------------------------------------
    """
    value = int(str(value)[:2])
    indurstry_code_key = [k for k, v in main_dict['코드'].items() if value in v][0]
        
    return main_dict['항목명'][indurstry_code_key]

In [31]:
# 업종중분류 만들기
df_add_middle_class = df.copy()
df_add_middle_class['업종중분류'] = df['산업코드1'].apply(add_middle_class)

In [32]:
# 업종대분류 만들기
df_add_main_class = df_add_middle_class.copy()
df_add_main_class['업종대분류'] = df_add_middle_class['산업코드1'].apply(add_main_class)

In [33]:
# 추가한 파생변수들이 있는 데이터프레임 내보내기
df_add_main_class.to_csv('./custom_data/feature_engineering_idurstry_code.csv', encoding='cp949', index=False)